In [21]:
import tensorflow

In [22]:
from tensorflow import keras

In [23]:
from keras import layers, models, optimizers, losses, metrics

In [24]:
!tar xf aclImdb_v1.tar.gz  

In [25]:
!rm -rf aclImdb/train/unsup

In [26]:
!ls aclImdb/train/

labeledBow.feat  pos		urls_neg.txt  urls_unsup.txt
neg		 unsupBow.feat	urls_pos.txt


In [27]:
import re
import string

def standardize(text):
    text = tensorflow.strings.lower(text)
    text = tensorflow.strings.regex_replace(text, '<br />', '')
    text = tensorflow.strings.regex_replace(text, '[%s]' % re.escape(string.punctuation), '')
    return text

In [28]:
standardize('aAbcd')

<tf.Tensor: shape=(), dtype=string, numpy=b'aabcd'>

In [29]:
'[%s]' % re.escape(string.punctuation)

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [30]:
re.escape(string.punctuation)

'!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~'

In [31]:
batch_size=50
seed=50

raw_train_ds = keras.utils.text_dataset_from_directory('aclImdb/train/', batch_size=batch_size, validation_split=0.2, seed=seed, subset='training')
raw_val_ds = keras.utils.text_dataset_from_directory('aclImdb/train/', batch_size=batch_size, validation_split=0.2, seed=seed, subset='validation')
raw_test_ds = keras.utils.text_dataset_from_directory('aclImdb/test/', batch_size=batch_size, validation_split=0.2, seed=seed, subset='validation')

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [32]:
max_features=10000
seq_len = 250

vec_layer = layers.TextVectorization(standardize=standardize, max_tokens=max_features, output_mode='int', output_sequence_length=seq_len)

In [33]:
def vec_text(text, label):
    text = tensorflow.expand_dims(text, -1)
    return vec_layer(text), label

In [34]:
train_text = raw_train_ds.map(lambda x, y: x)
vec_layer.adapt(train_text)

In [35]:
AUTOTUNE=tensorflow.data.AUTOTUNE

train_ds = raw_train_ds.map(vec_text).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.map(vec_text).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = raw_test_ds.map(vec_text).cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
embedding_dim = 16
model = models.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          160000    
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160017 (625.07 KB)
Trainable params: 160017 (625.07 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [37]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=metrics.BinaryAccuracy(threshold=0.0))

In [38]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10


2026-01-02 11:16:11.035808: I external/local_xla/xla/service/service.cc:168] XLA service 0x71577c0c12d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-01-02 11:16:11.035831: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2026-01-02 11:16:11.040934: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-01-02 11:16:11.060164: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
I0000 00:00:1767352571.117180    2704 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


400/400 [==============================] - 26s 61ms/step - loss: 0.6749 - binary_accuracy: 0.6825 - val_loss: 0.6458 - val_binary_accuracy: 0.7524
Epoch 2/10
400/400 [==============================] - 1s 3ms/step - loss: 0.5971 - binary_accuracy: 0.7810 - val_loss: 0.5598 - val_binary_accuracy: 0.7826
Epoch 3/10
400/400 [==============================] - 1s 3ms/step - loss: 0.5095 - binary_accuracy: 0.8231 - val_loss: 0.4867 - val_binary_accuracy: 0.8164
Epoch 4/10
400/400 [==============================] - 1s 3ms/step - loss: 0.4398 - binary_accuracy: 0.8508 - val_loss: 0.4348 - val_binary_accuracy: 0.8346
Epoch 5/10
400/400 [==============================] - 1s 3ms/step - loss: 0.3899 - binary_accuracy: 0.8652 - val_loss: 0.3997 - val_binary_accuracy: 0.8468
Epoch 6/10
400/400 [==============================] - 2s 4ms/step - loss: 0.3543 - binary_accuracy: 0.8757 - val_loss: 0.3755 - val_binary_accuracy: 0.8520
Epoch 7/10
400/400 [==============================] - 1s 3ms/step - loss:

In [39]:
model.evaluate(test_ds)

100/100 [==============================] - 0s 3ms/step - loss: 0.3305 - binary_accuracy: 0.8658


[0.33049070835113525, 0.8658000230789185]

In [46]:
model2 = models.Sequential([
    layers.Embedding(max_features, embedding_dim),
    layers.Bidirectional(layers.LSTM(32)), 
    layers.Dropout(0.5),
    layers.Dense(1)
])

model2.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=metrics.BinaryAccuracy(threshold=0.0))

In [47]:
history = model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
400/400 [==============================] - 25s 57ms/step - loss: 0.6280 - binary_accuracy: 0.6418 - val_loss: 0.4253 - val_binary_accuracy: 0.8198
Epoch 2/5
400/400 [==============================] - 8s 19ms/step - loss: 0.3423 - binary_accuracy: 0.8667 - val_loss: 0.3699 - val_binary_accuracy: 0.8644
Epoch 3/5
400/400 [==============================] - 8s 19ms/step - loss: 0.2557 - binary_accuracy: 0.9102 - val_loss: 0.4652 - val_binary_accuracy: 0.8086
Epoch 4/5
400/400 [==============================] - 7s 17ms/step - loss: 0.2168 - binary_accuracy: 0.9230 - val_loss: 0.4108 - val_binary_accuracy: 0.8628
Epoch 5/5
400/400 [==============================] - 7s 18ms/step - loss: 0.1716 - binary_accuracy: 0.9445 - val_loss: 0.6532 - val_binary_accuracy: 0.8382


In [48]:
model2.evaluate(test_ds)

100/100 [==============================] - 1s 8ms/step - loss: 0.6912 - binary_accuracy: 0.8314


[0.6912481784820557, 0.8313999772071838]

In [51]:
model3 = models.Sequential([
    layers.Embedding(max_features, embedding_dim),
    layers.Conv1D(32, 7, activation='relu'), # Fast like DNN, smart like RNN
    layers.GlobalMaxPooling1D(),
    layers.Dense(1)
])

model3.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=metrics.BinaryAccuracy(threshold=0.0))

history = model3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
400/400 [==============================] - 20s 46ms/step - loss: 0.5733 - binary_accuracy: 0.6977 - val_loss: 0.4355 - val_binary_accuracy: 0.7980
Epoch 2/5
400/400 [==============================] - 2s 6ms/step - loss: 0.3572 - binary_accuracy: 0.8457 - val_loss: 0.3467 - val_binary_accuracy: 0.8484
Epoch 3/5
400/400 [==============================] - 1s 4ms/step - loss: 0.2406 - binary_accuracy: 0.9078 - val_loss: 0.3357 - val_binary_accuracy: 0.8556
Epoch 4/5
400/400 [==============================] - 1s 3ms/step - loss: 0.1626 - binary_accuracy: 0.9439 - val_loss: 0.3509 - val_binary_accuracy: 0.8578
Epoch 5/5
400/400 [==============================] - 1s 3ms/step - loss: 0.1046 - binary_accuracy: 0.9689 - val_loss: 0.3814 - val_binary_accuracy: 0.8526


In [52]:
model3.evaluate(test_ds)

100/100 [==============================] - 0s 2ms/step - loss: 0.3709 - binary_accuracy: 0.8608


[0.37092533707618713, 0.86080002784729]